# Statistical tests 

Tests are performed on the year 2019.


1. Test for statistical significant difference in performance accuarcy between probabilistic climatology (EPC 0) and EPC window of size 2, 5, 10, 15, 20. Apply 2-sided Diebold Mariano test with spatial correction.


2. Test for "equal" forecast accuarcy between EPC 20 and EPC 15. Apply 2 1-sided Diebold Mariano test with spatial correction and equivalence margin 

In [6]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

from cartopy import config
import cartopy.crs as ccrs
import pickle

from scipy.stats import norm

import matplotlib.patches as mpatches
import pandas as pd

In [ ]:
# Read in data
epc0 = xr.open_mfdataset('D:/crps19_epc/crps19_0_*.nc4', concat_dim='time', combine='nested')
epc2 = xr.open_mfdataset('D:/crps19_epc/crps19_2_*.nc4', concat_dim='time', combine='nested')
epc5 = xr.open_mfdataset('D:/crps19_epc/crps19_5_*.nc4', concat_dim='time', combine='nested')
epc10 = xr.open_mfdataset('D:/crps19_epc/crps19_10_*.nc4', concat_dim='time', combine='nested')
epc15 = xr.open_mfdataset('D:/crps19_epc/crps19_15_*.nc4', concat_dim='time', combine='nested')
epc20 = xr.open_mfdataset('D:/crps19_epc/crps19_20_*.nc4', concat_dim='time', combine='nested')

In [ ]:
# Prepare data
tttime = pd.date_range(start='01/01/2019', end='12/31/2019')
epc0 = epc0.assign_coords(time = tttime)
epc2 = epc2.assign_coords(time = tttime)
epc5 = epc5.assign_coords(time = tttime)
epc10 = epc10.assign_coords(time = tttime) 
epc15 = epc15.assign_coords(time = tttime) 
epc20 = epc20.assign_coords(time = tttime) 

In [ ]:
# define function to compute teststatsitic for diebold mariano test
def teststatistic(epc1, epc2):
    Smean_F = epc1.mean(dim="time")
    Smean_G = epc2.mean(dim="time")
    difference = np.subtract(Smean_F,Smean_G)
    factor = np.sqrt(365)
    variance = np.square(np.subtract(epc1,epc2)).mean(dim="time")
    sigma = np.sqrt(variance)
    teststatistic = factor*np.divide(difference, sigma)
    tt = teststatistic.precipitationCal.values
    return(tt)

In [ ]:
# compute teststatistic
test15_0 = teststatistic(epc15, epc0)
np.save('test15_0.npy', test15_0)
test10_0 = teststatistic(epc10, epc0)
np.save('test10_0.npy', test10_0)
test20_0 = teststatistic(epc20, epc0)
np.save('test20_0.npy', test20_0)
test5_0 = teststatistic(epc5, epc0)
np.save('test5_0.npy', test5_0)
test2_0 = teststatistic(epc2, epc0)
np.save('test2_0.npy', test2_0)
test15_10 = teststatistic(epc15, epc10)
np.save('test15_10.npy', test15_10)
test20_15 = teststatistic(epc20, epc15)
np.save('test20_15.npy', test20_15)
test20_10 = teststatistic(epc20, epc10)
np.save('test20_10.npy', test20_15)

In [ ]:
variance = np.square(np.subtract(epc20,epc10)).mean(dim="time")
sigma = np.sqrt(variance).precipitationCal.values
np.save('sigma20_10.npy',sigma)

variance = np.square(np.subtract(epc20,epc15)).mean(dim="time")
sigma = np.sqrt(variance).precipitationCal.values
np.save('sigma20_15.npy',sigma)

variance = np.square(np.subtract(epc15,epc10)).mean(dim="time")
sigma = np.sqrt(variance).precipitationCal.values
np.save('sigma15_10.npy',sigma)

In [22]:
# 1. Run 2-sided diebold marinao test
teststatistics_list = ['2','5','10','15','20']
alpha = 0.05
for teststat in teststatistics_list:
    teststatname = 'C:/Users/walz/Documents/Work/Promotion/EPC/Scripts/Python/test'+teststat+'_0.npy'
    testval = np.load(teststatname)
    dimspace = testval.shape[0]*testval.shape[1]
    ds = dimspace/100
    pval = (1-norm.cdf(np.absolute(testval)))*2
    sign_percent = np.min(np.where((np.sort(pval.flatten()) <= np.arange(1,(dimspace+1))*alpha/(dimspace))==False))/(ds)   
    
    rejectval = np.where((np.sort(pval.flatten()) <= np.arange(1,(dimspace+1))*alpha/(dimspace))==False, 0, 1)
    indxpval = np.argsort(pval.flatten())
    
    a = np.column_stack((rejectval,indxpval))
    a = a[a[:,1].argsort()]
    a2 = np.where(a[:,0] == 1)
    testvalflt = testval.flatten()
    sup_percent = np.round(np.sum(testvalflt[a2] < 0)/ds,2)
    
    info = 'EPC 0 and EPC ' + teststat + ': significantly different at'
    infos = teststat + ' viewed superior at'
    print(info, np.round(sign_percent,2), '% of locations. EPC', infos, sup_percent, '% of locations')

EPC 0 and EPC 2: significantly different at 81.98 % of locations. EPC 2 viewed superior at 81.98 % of locations
EPC 0 and EPC 5: significantly different at 83.98 % of locations. EPC 5 viewed superior at 83.98 % of locations
EPC 0 and EPC 10: significantly different at 84.37 % of locations. EPC 10 viewed superior at 84.37 % of locations
EPC 0 and EPC 15: significantly different at 83.93 % of locations. EPC 15 viewed superior at 83.93 % of locations
EPC 0 and EPC 20: significantly different at 83.37 % of locations. EPC 20 viewed superior at 83.37 % of locations


In [73]:
#2. Run two 1-sided diebold mariano test
theta = 0.05
alpha = 0.05
days = 365
for teststat in ['20_15', '20_10', '15_10']:
    sigma = np.load('C:/Users/walz/Documents/Work/Promotion/EPC/Scripts/Python/sigma'+teststat+'.npy')
    testval = np.load('C:/Users/walz/Documents/Work/Promotion/EPC/Scripts/Python/test'+teststat+'.npy')
    thetacorrection = (theta*np.sqrt(days))/sigma
    test_l = testval + thetacorrection
    test_u = thetacorrection - testval
    dimspace = testval.shape[0]*testval.shape[1]
    ds = dimspace / 100
    compare = np.arange(1, dimspace+1)

    pval_l = 1-norm.cdf(test_l.flatten())
    pval_u = 1-norm.cdf(test_u.flatten())
    out_l = np.where(np.sort(pval_l) <= compare*alpha/dimspace, 1, 0)
    out_u = np.where(np.sort(pval_u) <= compare*alpha/dimspace, 1, 0)
    indx1 = np.argsort(pval_l) 
    indx2 = np.argsort(pval_u)
    a = np.column_stack((out_l,indx1))
    a = a[a[:,1].argsort()]
    a2 = np.where(a[:,0] == 1)
    b = np.column_stack((out_u,indx2)) 
    b = b[b[:,1].argsort()]
    b2 = np.where(a[:,0] == 1)
    percent_test = np.round(len(np.unique(np.concatenate([a2,b2], axis=1)))/ds,2)
    print(teststat + ': equivalence at ' , percent_test ,'% of locations')

20_15: equivalence at  99.78 % of locations
20_10: equivalence at  93.45 % of locations
15_10: equivalence at  97.37 % of locations
